In [2]:
import os, re 
import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담기
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))

데이터 크기: 187088


In [3]:
# ====데이터 전처리====

# 정규화를 이용한 정제 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>'
    
    return sentence

corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0: continue

    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

corpus[:3]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>']

In [4]:
# corpus를 텐서로 변환
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    # corpus -> tensor 변환
    tensor = tokenizer.texts_to_sequences(corpus)
    #print(len(tensor))
    # 길이가 15이상이면 제외
    for index, value in enumerate(tensor):
        if len(tensor[index]) >= 15 :
            del tensor[index]
    #print(len(tensor))
    # 시퀀스 데이터 조정
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

# 소스와 타켓으로 분리
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]  

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=10)

In [6]:
#corpus 텐서를 데이터셋 객체로 변환
# src_input->enc_train
# tgt_input->dec_train
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로 데이터셋을 만듦
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) 
dataset


<BatchDataset shapes: ((100, 346), (100, 346)), types: (tf.int32, tf.int32)>

In [12]:
# 학습시키기
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 50
hidden_size = 300
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(100, 346, 12001), dtype=float32, numpy=
array([[[ 8.10096972e-05,  1.00796851e-05,  1.16046045e-04, ...,
         -7.68064274e-06, -2.66981988e-05, -7.07694635e-05],
        [ 1.46241131e-04,  8.66715927e-05,  2.58704618e-04, ...,
         -2.69411667e-05, -8.50036158e-05, -1.21281926e-04],
        [ 7.58928436e-05,  5.89185765e-06,  2.98581173e-04, ...,
         -4.92488180e-05, -7.60763869e-05, -1.99682501e-04],
        ...,
        [-1.00838079e-03,  3.65082378e-04,  2.99721374e-04, ...,
          1.04058860e-03,  2.34526768e-03,  1.00636412e-03],
        [-1.00838090e-03,  3.65082349e-04,  2.99721316e-04, ...,
          1.04058860e-03,  2.34526745e-03,  1.00636424e-03],
        [-1.00838102e-03,  3.65082233e-04,  2.99721432e-04, ...,
          1.04058860e-03,  2.34526745e-03,  1.00636424e-03]],

       [[ 8.10096972e-05,  1.00796851e-05,  1.16046045e-04, ...,
         -7.68064274e-06, -2.66981988e-05, -7.07694635e-05],
        [ 5.48396929e-05,  8.36044364e-06,  

In [13]:
model.summary()

Model: "text_generator_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  600050    
_________________________________________________________________
lstm_8 (LSTM)                multiple                  421200    
_________________________________________________________________
lstm_9 (LSTM)                multiple                  721200    
_________________________________________________________________
dense_4 (Dense)              multiple                  3612301   
Total params: 5,354,751
Trainable params: 5,354,751
Non-trainable params: 0
_________________________________________________________________


In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
1241/1241 [==============================] - 637s 512ms/step - loss: 0.2272
Epoch 2/10
1241/1241 [==============================] - 642s 517ms/step - loss: 0.1341
Epoch 3/10
1241/1241 [==============================] - 644s 519ms/step - loss: 0.1274
Epoch 4/10
1241/1241 [==============================] - 646s 521ms/step - loss: 0.1228
Epoch 5/10
1241/1241 [==============================] - 650s 523ms/step - loss: 0.1282
Epoch 6/10
1241/1241 [==============================] - 650s 524ms/step - loss: 0.1185
Epoch 7/10
1241/1241 [==============================] - 651s 524ms/step - loss: 0.1150
Epoch 8/10
1241/1241 [==============================] - 651s 524ms/step - loss: 0.1116
Epoch 9/10
1241/1241 [==============================] - 651s 525ms/step - loss: 0.1083
Epoch 10/10
1241/1241 [==============================] - 652s 525ms/step - loss: 0.1050


In [21]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [25]:
generate_text(model, tokenizer, init_sentence="<start> i call", max_len=20)

'<start> i call you in the world <end> '